In [1]:
#Selection of a few traces for the further alignments computation (example of PM4)

import pm4py
import os
import pm4py
import psutil
from pm4py import util

log = pm4py.read_xes("C:/Users/I559643/Documents/Thesis/Models and Logs/PM 8 with 41act/log 100 tracespm8.xes")
lst = ['case_7','case_51', 'case_11','case_24','case_62','case_30','case_74','case_47']
log_filtered = log[log['case:concept:name'].isin(lst)]
pm4py.write_xes(log_filtered, "C:/Users/I559643/Documents/Thesis/Models and Logs/log 100 tracespm8_few.xes")

C:\Users\I559643\Miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|███████████████████████████████████████████████| 8/8 [00:00<00:00, 1998.12it/s]


In [2]:
#Alignments computations

from pm4py import util
from pm4py.algo.conformance import alignments as ali
from pm4py.algo.conformance.alignments.petri_net.variants.state_equation_a_star import Parameters
from pm4py.objects import log as log_lib
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.petri_net.importer import importer as petri_importer
from pm4py.objects.petri_net.utils.align_utils import pretty_print_alignments
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.visualization.petri_net.variants import alignments


def align(trace, net, im, fm, model_cost_function, sync_cost_function):
    #trace_costs = list(map(lambda e: 1000, trace))
    params = dict()
    params[util.constants.PARAMETER_CONSTANT_ACTIVITY_KEY] = log_lib.util.xes.DEFAULT_NAME_KEY
    params[Parameters.PARAM_MODEL_COST_FUNCTION] = model_cost_function
    #params[Parameters.PARAM_TRACE_COST_FUNCTION] = trace_costs
    params[Parameters.PARAM_SYNC_COST_FUNCTION] = sync_cost_function
    return ali.petri_net.algorithm.apply_trace(trace, net, im, fm, parameters=params,
                                   variant=ali.petri_net.algorithm.VERSION_STATE_EQUATION_A_STAR)

def execute_script():
    log = xes_importer.apply("C:/Users/I559643/Documents/Thesis/Models and Logs/PM 8 with 41act/log 100 tracespm8_few.xes")
    net, marking, fmarking = petri_importer.apply('C:/Users/I559643/Documents/Thesis/Models and Logs/PM 8 with 41act/Test 42 actiivit.pnml')
         
    model_cost_function = dict()
    sync_cost_function = dict()
    for t in net.transitions:
        if t.label is not None:
            model_cost_function[t] = 1000
            sync_cost_function[t] = 0
        else:
            model_cost_function[t] = 1
            
    alignments = []

    log2 = pm4py.convert_to_dataframe(log)
    list1 =  log2["case:concept:name"].values.tolist()
    

    test_list = []      # sometimes retrieved cases are randomly placed in a list, but we need to know the exact trace for which alignment is computed 
                        #thus (below in the code) print alignment and corresponding trace id.
    for el in list1:    
        if el not in test_list:   
            test_list.append(el)  
    #print(test_list) 


    for trace in log:
        alignments.append(align(trace, net, marking, fmarking, model_cost_function, sync_cost_function))
#         print(alignments)

    
    for element in list(zip(test_list, alignments)):
        print(f"The ID is '{element[0]}' and the alignment = {element[1]}", trace)
    
    pretty_print_alignments(alignments)



    
if __name__ == '__main__':
    execute_script()

parsing log, completed traces :: 100%|█████████████████████████████████████████████████| 8/8 [00:00<00:00, 1148.22it/s]

The ID is 'case_74' and the alignment = {'alignment': [('Activity A', 'Activity A'), ('>>', None), ('Activity C', 'Activity C'), ('Activity E', 'Activity E'), ('Activity D', 'Activity D'), ('Activity F', 'Activity F'), ('>>', None), ('Activity B', 'Activity B'), ('Activity G', 'Activity G'), ('Activity I', 'Activity I'), ('>>', None), ('Activity I', 'Activity I'), ('Activity H', 'Activity H'), ('Activity J', 'Activity J'), ('Activity K', 'Activity K'), ('Activity L', 'Activity L'), ('Activity S', 'Activity S'), ('Activity T', 'Activity T'), ('Activity U', 'Activity U'), ('Activity M', 'Activity M')], 'cost': 3, 'visited_states': 20, 'queued_states': 65, 'traversed_arcs': 66, 'lp_solved': 1, 'fitness': 1.0, 'bwc': 186002} {'attributes': {'concept:name': 'case_24'}, 'events': [{'concept:name': 'Activity A', 'time:timestamp': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'case:concept:name': 'case_24'}, '..', {'concept:name': 'Activity M', 'time:timestamp': datetime.d


C:\Users\I559643\Miniconda3\lib\site-packages\pm4py\utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [ ]:
import pm4py


dataframe = pm4py.read_xes("C:/Users/I559643/Documents/Thesis/Models and Logs/process with evolution/Log for pm 1_1.xes")

list1 =  dataframe["case:concept:name"].values.tolist()
    

test_list = []     
for el in list1:    
    if el not in test_list:   
        test_list.append(el) 


net, im, fm = pm4py.discover_petri_net_inductive(dataframe, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
alignments_diagnostics = pm4py.conformance_diagnostics_alignments(dataframe, net, im, fm, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
for element in list(zip(test_list, alignments_diagnostics)):
    print(f"The ID is '{element[0]}' and the alignment = {element[1]}", alignments_diagnostics)
#print(f"The ID is '{element[0]}' and the alignment = {element[1]}", alignments_diagnostics)